# yveCRV Reward Claims
> "How do users handle yveCRV reward claims?"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Curve, Yearn]
- hide: false  


In [1]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


dplyr.summarise.inform,[RTYPES.NILSXP]


In [12]:
#hide
%%R
#Grab base query from Flipside
df_yveCRV = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/8dc06609-8d0e-4f5d-b754-e61082b336fa/data/latest', simplifyDataFrame = TRUE)
df_claims = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/180e0b1f-5d73-4beb-b4cb-8399dba03b47/data/latest', simplifyDataFrame = TRUE)

#fix the column names
names(df_yveCRV)<-tolower(names(df_yveCRV))
names(df_claims)<-tolower(names(df_claims))

#Change the date to date format
df_yveCRV$date <- as.Date(parse_datetime(df_yveCRV$date))
df_claims$date <- as.Date(parse_datetime(df_claims$date))
df_claims$block_timestamp <- parse_datetime(df_claims$block_timestamp)

#create a date sequence from min date to max date
full_date_range <- tibble(date = seq(min(df_yveCRV$date), max(df_yveCRV$date), by = "days"))

#join in the df_yveCRV frame
df_yveCRV <- full_date_range %>%
  left_join(df_yveCRV, by=c('date'))

#fill the na prices
df_yveCRV <- df_yveCRV %>%
    replace_na(list(threecrv_deposit = 0, yvecrv_minted = 0))

#df_yveCRV tokens on issue
df_yveCRV <- df_yveCRV %>%
  arrange(date) %>%
  mutate(yveCRV_on_issue = cumsum(yvecrv_minted))

#Grab yveCRV prices from coingecko
cg_api = paste('https://api.coingecko.com/api/v3/coins/vecrv-dao-yvault/market_chart/range?vs_currency=usd&from=',
               as.numeric(as.POSIXct(min(df_yveCRV$date))),
               '&to=',
               as.numeric(as.POSIXct(max(df_yveCRV$date))),
               sep = "")


yveCRV_prices = fromJSON(cg_api, simplifyDataFrame = TRUE, flatten = TRUE)

yveCRV_prices <- as_tibble(yveCRV_prices$prices) %>%
  mutate(date = as.Date(as.POSIXct(V1/1000, origin="1970-01-01")),
         yveCRV_price = V2) %>%
  select(date, yveCRV_price)

#join back into the main df
df_yveCRV <- df_yveCRV %>%
  left_join(yveCRV_prices, by = c('date'))

#Grab CRV prices from coingecko
cg_api = paste('https://api.coingecko.com/api/v3/coins/curve-dao-token/market_chart/range?vs_currency=usd&from=',
               as.numeric(as.POSIXct(min(df_yveCRV$date))),
               '&to=',
               as.numeric(as.POSIXct(max(df_yveCRV$date))),
               sep = "")


CRV_prices = fromJSON(cg_api, simplifyDataFrame = TRUE, flatten = TRUE)

CRV_prices <- as_tibble(CRV_prices$prices) %>%
  mutate(date = as.Date(as.POSIXct(V1/1000, origin="1970-01-01")),
         CRV_price = V2) %>%
  select(date, CRV_price)


#define ROI as the annualised 7 day vault return - assume 52 weeks/year
#ROI is 3CRV received over tokens on issue
df_yveCRV <- df_yveCRV %>%  
#  drop_na(price) %>%
  mutate(yveCRV_7day_ROI = if_else(threecrv_deposit == 0, NA_real_, threecrv_deposit / (yveCRV_on_issue*yveCRV_price)) * 52 * 100)  %>%
  fill(yveCRV_7day_ROI, .direction = "down")

rm(list = c('full_date_range','yveCRV_prices', 'cg_api', 'CRV_prices'))


#Create a week field
df_claims <- df_claims %>%
  mutate(week = floor_date(date, unit='week'))
df_yveCRV <- df_yveCRV %>%
  mutate(week = floor_date(date, unit='week'))


#For claims, create a % of claim gas fee field
df_claims <- df_claims %>%
  mutate(gas_percentage_of_claim = fee_usd / amount * 100,
         net_return = amount - fee_usd)

#Create roll-up buckets
df_claims <- df_claims %>%
  mutate(claim_lost_to_gas = case_when (
    net_return < 0 ~ "Negative Return",
    net_return >0 & gas_percentage_of_claim > 50 ~ "50-100% Loss",
    net_return >0 & gas_percentage_of_claim > 20 ~ "20-50% Loss",
    net_return >0 & gas_percentage_of_claim > 10 ~ "10-20% Loss",
    net_return >0 & gas_percentage_of_claim > 5 ~ "5-10% Loss",
    net_return >0 & gas_percentage_of_claim > 1 ~ "1-5% Loss",
    TRUE ~ "< 1% Loss",
  )) 

#fix bucket ordering
df_claims$claim_lost_to_gas = parse_factor(df_claims$claim_lost_to_gas,
                                           levels = c("Negative Return", "50-100% Loss", 
                                                      "20-50% Loss", "10-20% Loss",
                                                      "5-10% Loss", "1-5% Loss",
                                                      "< 1% Loss"),
                                           ordered = TRUE)
#roll up & summarise
roll_up_table <- df_claims %>%
  group_by(claim_lost_to_gas) %>%
  summarise(no_of_claims = n(),
            gas_spent_usd = round(sum(fee_usd, na.rm = TRUE),0),
            gas_spent_eth = round(sum(tx_fee),2),
            amount_claimed = round(sum(amount),0)
  ) %>%
  arrange(claim_lost_to_gas) %>%
  mutate(cumu_claims = cumsum(no_of_claims),
         cumu_claims_percent = cumu_claims / sum(no_of_claims) * 100)

#create the total row
total_row <- roll_up_table %>%
  summarise(claim_lost_to_gas = 'Total',
            no_of_claims = sum(no_of_claims),
            gas_spent_usd = sum(gas_spent_usd),
            gas_spent_eth = sum(gas_spent_eth),
            amount_claimed = sum(amount_claimed),
            cumu_claims = sum(no_of_claims),
            cumu_claims_percent = 100
            )
  
#summarise by week
weekly_claims <- df_claims %>%
  group_by(week) %>%
  summarise(no_of_claims = n(),
            gas_spent_usd = round(sum(fee_usd, na.rm = TRUE),0),
            gas_spent_eth = round(sum(tx_fee),2),
            amount_claimed = round(sum(amount),0)
  ) %>%
  left_join(df_yveCRV %>% group_by(week) %>% summarise(avail_to_claim = sum(threecrv_deposit, na.rm = TRUE)), by = "week") %>%
  mutate(percent_claimed = amount_claimed / avail_to_claim * 100,
         cumu_claimed = cumsum(amount_claimed),
         cumu_avail = cumsum(avail_to_claim),
         cumu_percent = cumu_claimed / cumu_avail * 100
         )
  
#fixup week field
weekly_claims$week <- as_datetime(weekly_claims$week)

#set up claims % table
claims_percent_table <- weekly_claims %>% 
  mutate(cumu_percent_unclaimed = 100 - cumu_percent) %>% 
  select(week, cumu_percent, cumu_percent_unclaimed) %>%
  rename('Percent Claimed' = cumu_percent,
         'Percent Unclaimed' = cumu_percent_unclaimed) %>%
  pivot_longer(!week, names_to="measure", values_to = "percentage")

# Yearn yVaults and the yveCRV Vault

## Yearn yVaults
Yearn is a DeFi protocol which automates yield farming.  Users have tokens which they want to hold - Yearn puts those tokens to work by finding the best yield farming opportunities across DeFi.  It does this in a gas efficient way for the user, so even small deposits can get decent returns over time.

Yearn users deposit their tokens into yVaults, and receive a token in return which is proportional to their share of the vault capital.  A yVault is a smart contract with one ore more Strategies sitting behind it.  The Strategies are the yield-farming recipes which are created by clever humans (Strategists) and monitored & managed by bots (Keepers).  The yVault contains logic which automatically allocates the vault deposits to whichever combination of Strategies gives the best return for the users.  The rewards from the yield farming accrue into the vault, so the value of the vault token is always increasing.  When a user withdraws from the yVault, they get more tokens than they deposited.  This additional amount is the yield the vault has earned on their behalf.

Each yVault is structured around a particular underlying token - there are vaults for Eth, USDC, WBTC and many others.  Users can deposit in the yVault native token, or they can deposit using any other token & take advantage of Zaps.  Zaps are smart contracts which take an input token and swap it for the underlying token in a gas efficient manner.  The swap may occur via a number of dexs or dex aggregators, but this is abstracted away for the user.  There is a similar feature when withdrawing - the user can withdraw the underlying token from a yVault, or choose to receive their funds in ETH, WBTC, DAI, USDC or USDT.  It's important to know that whatever token the user deposits  or withdraws, they maintain price exposure to the *underlying token* of the vault whilst deposited. 


## yveCRV yVault
Now the veCRV-DAO yVault (also known as the yveCRV Vault) is a little different to the others.  It starts with the CRV token, the governance & reward token from [Curve](https://curve.fi).  Curve is a dex which specialises in stableswaps - swaps between tokens which have approximately the same value.  Examples are ETH/stETH or swaps between dollar pegged stablecoins.  Curve has optimised their swap code to make these swaps efficient from both a liquidity impact and fee perspected - see this [post](https://scottincrypto.github.io/analytics/curve/2021/09/19/Curve_Stableswaps.html) for a further exploration of Curve & stablecoin swaps.

The CRV token has voting rights in the Curve DAO which makes decisions on the Curve protocol - things like fees, LP rewards, swap parameters and pools launched.  In some Curve pools, liquidity providers receive CRV tokens to incentivise liquidity in the pools.  CRV tokens are also available on the open market.  To encourage users to stay as CRV hodlers, there is a facility to lock CRV tokens into the CRV DAO for a fixed period of up to 4 years.  Users receive veCRV tokens (voting escrow Curve Tokens) for doing this, and more tokens are received the longer the locking period.  veCRV holders can still participate in governance voting, they receive 50% of Curve trading fees and they qualify for boosted rewards (up to 2.5x) when they provide liquidity in Curve.

The fees generated for veCRV holders are collected in the form of 3CRV tokens (shares in the [Curve tripool](https://curve.fi/3pool)), which can be redeemed for stablecoins if desired.  Fee distribution for veCRV holders happens weekly and users need to collect these manually and pay the gas cost for the transactions.

In true Yearn fashion there is a vault & a strategy to maximise the returns from this CRV locking process.  This is the yveCRV yVault and it's different to the other vaults in that you *can't withdraw your tokens*.  Yearn takes CRV tokens and locks them with the CRV DAO for the maximum 4 year period and continually renews this lock.  This maximises the veCRV returns to the yVault.  In addition, all Yearn vaults send 10% of earned CRV into this vault for additional boost.  The returns to the users are in the form of the 3CRV tokens earned by the veCRV - like the CRV staking contract, these are collectable weekly as an income stream, and must be collected manually.  yVault depositors receive yveCRV-DAO tokens as their share in the vault.

# Weekly Claiming of 3CRV Tokens

As mentioned above, users must claim the 3CRV rewards from the yveCRV vault manually.  This is an on-chain transaction and requires expenditure of gas to do so.  This gives two barriers to claiming rewards - users must remember to do it weekly, and they must make a choice as to whether the gas expenditure is worth the claim.  It should be noted that pending claims accrue and are not lost - a user can claim less frequently than monthly which would result in less overal gas spent.  The chart below shows the number of claim events each week on the yveCRV vault.  As we can see from the chart, the number of claims ranges between 18 and 72 (averaging 20-30)  per week for the period up to June 2020.  Then the average number increases to 40 per month for the next 6 weeks before dropping considerably.  This increase, then decrease corresponds with the decrease in yveCRV pricing relative to curve discussed in this post on the [impact of yvBoost on yveCRV](https://scottincrypto.github.io/analytics/curve/yearn/2021/09/29/yvBoost-and-yveCRV-Vaults.html#Impact-of-Pricing).  It appears that users claimed their rewards, then sold yveCRV on the open market in order to deploy their CRV tokens elsewhere - perhaps to yvBoost where the claim process is automated, or to other sources of yield like [Convex](https://convexfinance.com).  Now it appears there are far fewer holders regularly claiming tokens - only around 10 per week.

In [11]:
#@title
#hide_input
#Plot no of claims each week
df_p = %R weekly_claims %>% filter(week < max(week))
fig = px.bar(df_p
             , x = "week"
             , y = "no_of_claims"
             #, color = 'claim_lost_to_gas'
             , labels=dict(week="Week", no_of_claims="No of Claims")
             , title= "Number of Claims on yveCRV per Week"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='No of Claims')
fig.update_xaxes(title_text=None)
fig.show()

# Rewards Claimed & Unclaimed

As an extension of how many users claim, it's worth exploring *how much* of the potential rewards are claimed by users.  We mentioned the barriers to claiming earlier - let's see what sort of impact they have.  The chart below shows the percentage of claimed & unclaimed 3CRV in the yvCRV vault.  The numbers are the cumulative percentages from the beginning of the vault operation.  We see from the chart a that currently only 70% of the rewards available have been claimed.  This has been as low as 34% back in April 2021.  Yearn saw a clear need to automate the claim process and the yvBoost initiative was the result.  The yvBoost vault accepts yveCRV-DAO tokens from yveCRV, and automatically claims the rewards each week on behalf of the users, swapping the 3CRV rewards for more yveCRV-DAO to increase the value locked in the vault.

In [15]:
#hide_input
#Plot the percentages
df_rel_debt = %R claims_percent_table
fig = px.area(df_rel_debt
              , x="week"
              , y="percentage"
              , color="measure"
              , template="simple_white", width=800, height=800/1.618
              , title= "Cumulative Percentage of Rewards Claimed in yveCRV Vault")
fig.update_yaxes(title_text='% of Available Claims')
fig.update_xaxes(title_text=None)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.90,
    xanchor="right",
    x=0.99
    
))
fig.update_layout(legend_title_text=None)

fig.show()

In [23]:
#hide_input
#Display summary table
%%R 
summary_table <- roll_up_table %>% 
  bind_rows(total_row) %>%
  select(claim_lost_to_gas, no_of_claims, amount_claimed, gas_spent_usd, gas_spent_eth) %>%
  rename("Claim Lost to Gas" = claim_lost_to_gas,
         "Number of Claims" = no_of_claims,
         "Amount Claimed (USD)" = amount_claimed,
         "Gas Spent (USD)" = gas_spent_usd,
         "Gas Spent (ETH)" = gas_spent_eth
         )
  
  total_table <- total_row %>% 
  select(no_of_claims, amount_claimed, gas_spent_usd, gas_spent_eth) %>%
  mutate("Percentage Gas Spend" = gas_spent_usd / amount_claimed * 100) %>%
  rename(
         "Number of Claims" = no_of_claims,
         "Amount Claimed (USD)" = amount_claimed,
         "Gas Spent (USD)" = gas_spent_usd,
         "Gas Spent (ETH)" = gas_spent_eth
         )


#How Much Gas Has Been Spent on Claims?

It was mentioned above that each claim requires an on-chain transaction and expenditure of gas.  Over the history of the yveCRV vault, users have spent just over 8 ETH on reward claims, worth over USD 16k when valued with the ETH price at the time of claim.  This is a small percentage of the rewards claimed however - around 0.77%.  We will look further into the detail of these claims and the gas costs to see whether gas has an impact on the overall returns of certain claims.

In [24]:
#hide_input
%R total_table

,Number of Claims,Amount Claimed (USD),Gas Spent (USD),Gas Spent (ETH),Percentage Gas Spend
1,1118,2172537.0,16660.0,8.07,0.766845


#  What Portion of Rewards are lost to Gas Costs?

To see if users are making sound decisions when claiming rewards from the yveCRV vault, we will look at how much the gas costs are in relation to the amount claimed.  We have simplified the dataset by grouping into a number of categories - firstly, was the amount claimed less than the gas cost, and if not, what percentage of the claim transaction did the gas represent.  The results are summarised below - how many transactions were in each category of loss.  Here we can see that 200 claims have been made for yveCRV rewards where the claimed amount was *less* than the gas spent.  The user made a negative return from the transaction.  Similarly, there were 76 claims where the user lost between 50-100% of the claimed amount to gas costs, and so on for the graph.  The largest category appears to be the 1-5% range.  This is interesting - users often shop around to find the very best APR for their funds, but are prepared to throw away a significant percentage in transaction costs to get it.

In [7]:
#@title
#hide_input
#Plot transaction count by bucket
df_p = %R roll_up_table
fig = px.bar(df_p
             , x = "claim_lost_to_gas"
             , y = "no_of_claims"
             , color = 'claim_lost_to_gas'
             , labels=dict(claim_lost_to_gas="Claim Loss", no_of_claims="No of Claims")
             , title= "Count of Claims by Claim Loss to Gas Fees"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='No of Claims')
fig.update_xaxes(title_text=None)
fig.show()

The graph below presents the same data as above, but as a cumulative percentage.  From this we can deduce:
- 18% of claim transactions were negative in value
- 25% of claim transactions had a greater than 50% loss
- 59% of claim transactions lost more than 5% to gas
- Only 17% of transactions had a loss less than 1%

It appears, on balance, that users are not making sound decisions on reward claims from the yveCRV vault.  Users face a choice when claiming smaller rewards.  Either:  
- Take the money now & pay a high portion in fees
- Try to claim in a time with lower gas costs
- Wait a few more weeks until rewards have accrued to make the gas cost less significant

It appears from the data that the opportunity cost for users of leaving the rewards unclaimed in yveCRV is greater than having the funds available right away.  For the users losing large percentages, or more than the total reward amount, perhaps there is a knowledge gap in the way gas fees work on Ethereum.

In [8]:
#@title
#hide_input
#Plot cumulative transaction count by bucket
df_p = %R roll_up_table
fig = px.bar(df_p
             , x = "claim_lost_to_gas"
             , y = "cumu_claims_percent"
             , color = 'claim_lost_to_gas'
             , labels=dict(claim_lost_to_gas="Claim Loss", cumu_claims_percent="Cumulative % of Claims")
             , title= "Cumulative % of Claims by Claim Loss to Gas Fees"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='Cumulative % of Claims')
fig.update_xaxes(title_text=None)
fig.show()

The following table shows the claims, amount claimed and gas spends for each of the categories above.  Most of the value claimed is claimed at less than 1% loss due to gas fees - this is the group of users with large deposits in yveCRV earning large rewards.  It's interesting that the highest gas expenditure overall came from the negative return group, again suggesting a lack of understanding of gas pricing & transaction pricing mechanisms.

In [25]:
#hide_input
%R summary_table

,Claim Lost to Gas,Number of Claims,Amount Claimed (USD),Gas Spent (USD),Gas Spent (ETH)
1,Negative Return,200,566.0,3446.0,1.80
2,50-100% Loss,76,1727.0,1198.0,0.54
3,20-50% Loss,119,4791.0,1480.0,0.78
4,10-20% Loss,131,13409.0,1832.0,0.81
5,5-10% Loss,129,23938.0,1673.0,0.87
6,1-5% Loss,270,140589.0,3185.0,1.58
7,< 1% Loss,193,1987517.0,3846.0,1.69
8,Total,1118,2172537.0,16660.0,8.07


# Conclusions

The yveCRV vault is a special one in the Yearn galaxy - it's the only yVault that doesn't support withdrawals so rewards must be claimed manually.  We have seen that the claim process is not effective for all users.  Only 70% of the historical vault rewards have been claimed.  Many users have made claims for rewards with a negative return due to gas costs, and many make claims that lose a large portion of their reward to gas.  Thankfully the Yearn team have addressed this with the yvBoost vault, where users get the same underlying exposure to CRV locking but the reward process is automated.  Since the launch of yvBoost, we have seen the number of weekly reward claims drop significantly, indicating that users have migrated away from the yveCRV by trading their tokens on the open market.

- All on-chain data sourced from the curated tables at [Flipside Crypto](https://flipsidecrypto.com)
- Pricing data sourced from [Coingecko](https://coingecko.com)